# Introduction and Basics of Chatbots and Conversational Agents

In this notebook, we work on the following topics:
- A direct API call from OpenAI to perform chat completions
- Chat completions with Langchain APIs
- Using prompt template to create custom prompt
- Using output parser to extract key information from the completion

**Use cases:**
- Technical support as a chatbot technician.
- Entity extraction from cusomer travel query.
 

In [1]:
%load_ext autoreload
%autoreload 2
import os
import pprint
import sys
module_dir = os.path.abspath('../../src')  # Gets the absolute path to the src directory
sys.path.append(module_dir)
from helper_functions import llm_completion

In [52]:
# A function for printing nicely
def nprint(text, indent=2):
    pp = pprint.PrettyPrinter(indent=indent)
    pp.pprint(text)

# Loading Parameters

In [3]:
modelID = "gpt-3.5-turbo"

# Technical support as a chatbot technician.
Here the LLM chatbot gives technical support to the customer to fix it's problem.  
We use system context as well as answering styles to customize the chatbot.


## API call from OpenAI

In [54]:
question = "What do I do if my screen freezes?"
completion = llm_completion(question, model=modelID)
nprint(completion)

('If your screen freezes, you can try the following steps to resolve the '
 'issue:\n'
 '\n'
 '1. Wait for a few moments: Sometimes the screen may freeze temporarily, so '
 'give it a few seconds to see if it resolves on its own.\n'
 '\n'
 "2. Restart your device: If waiting doesn't work, try restarting your device. "
 'This can help refresh the system and resolve any temporary glitches causing '
 'the freeze.\n'
 '\n'
 "3. Check for software updates: Make sure your device's operating system and "
 'apps are up to date. Sometimes freezes can occur due to outdated software.\n'
 '\n'
 '4. Close any unresponsive apps: If a specific app is causing the freeze, try '
 'closing it by using the task manager or force quitting the app.\n'
 '\n'
 '5. Remove any peripherals: If you have any external devices connected to '
 'your device, such as a USB drive or external monitor, try disconnecting them '
 'to see if they are causing the issue.\n'
 '\n'
 '6. Perform a hard reset: If the screen remains

## Working on the completion Style and System

In [55]:
sys_content = "You are an expert technician who gives exact, clear, and short steps to solving a problem."
completion = llm_completion(question, model=modelID, sys_content=sys_content)
nprint(completion)

('1. Try pressing Ctrl + Alt + Delete on Windows or Command + Option + Escape '
 'on Mac to open the Task Manager or Force Quit window.\n'
 "2. If that doesn't work, try restarting your computer by holding down the "
 'power button until it shuts off, then turning it back on.\n'
 '3. If the issue persists, check for any software updates or run a virus scan '
 'to rule out any potential software issues.\n'
 '4. If the problem continues, consider seeking help from a professional '
 'technician to diagnose and fix the underlying cause of the freeze.')


Now lets change the style of the answer

In [56]:
style = "The answer must be understabdable for a 5 year old child."
prompt = f"""{question} 
{style}
"""
print(prompt)

completion = llm_completion(prompt, model=modelID, sys_content=sys_content)
nprint(completion)

What do I do if my screen freezes? 
The answer must be understabdable for a 5 year old child.

('1. Press and hold the power button on your device.\n'
 '2. Wait for a few seconds until the screen turns off.\n'
 '3. Turn on your device again by pressing the power button once.')


Let's make it even more simpler for the child

In [57]:
sys_content = "You are a cool 5 years old child with funny sense of humor."

completion = llm_completion(prompt, model=modelID, sys_content=sys_content)
nprint(completion)

('If your screen freezes, just give it a little tickle by pressing the restart '
 "button or asking a grown-up to help you turn it off and on again. It's like "
 'giving your screen a little nap to wake it up feeling refreshed!')


Let's make it even more creative by increasing the tempreature of the LLM model to 2.0.

In [ ]:
completion = llm_completion(prompt, model=modelID, sys_content=sys_content, temperature=2.0)
nprint(completion)

# Using Langchain APIs to perform chat completions
Now we perform the above chat completions using Langchain modules which are more simpler and easier to use.  
We use prompt template to create the prompt with different style.

In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [5]:
QAchat = ChatOpenAI(temperature=0, model=modelID)
template = """{sys_content}
Answer the following question:  
{question} 
Style: {style}
"""
prompt_template = ChatPromptTemplate.from_template(template)
prompt_template.pretty_print()
print(prompt_template.messages[0].prompt.input_variables)

================================ Human Message =================================

{sys_content}
Answer the following question:  
{question} 
Style: {style}

['question', 'style', 'sys_content']


In [61]:
prompt = prompt_template.format_messages(
                    sys_content = sys_content,
                    style=style,
                    question=question)
print(prompt)
print("The formated prompt is:\n")
nprint(prompt[0].content)

[HumanMessage(content='You are a cool 5 years old child with funny sense of humor.\nAnswer this following question:  \nWhat do I do if my screen freezes? \nStyle: The answer must be understabdable for a 5 year old child.\n')]
The formated prompt is:

('You are a cool 5 years old child with funny sense of humor.\n'
 'Answer this following question:  \n'
 'What do I do if my screen freezes? \n'
 'Style: The answer must be understabdable for a 5 year old child.\n')


In [62]:
completion = QAchat(prompt)
nprint(completion.content)

('If your screen freezes, you can try turning it off and then back on again. '
 'Just like when you need a nap to feel better, sometimes your screen needs a '
 'little rest too!')


# Using output parser
Here we use output parser to extract key information from the completion.  
We need this information to be extracted as a JSON object, so we can use it as a dictionary.
So, We define a custom format instructions to let LLM know what information must be extracted from the completion.

**Use case**: Travel and Hospitality


In [23]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

Here is a sample query from the user from which we extract the key information about the trip.

In [12]:
customer_query = """\
We're planning a family vacation to Hawaii in December. \
We need recommendations for family-friendly hotels and activities suitable for kids. \
Also, any travel advisories we should be aware of?\
Our budget for the entire trip is around $5000.
"""

We use ResponseSchema to define how the key information must be extracted from the completion.

In [34]:
destination_schema = ResponseSchema(name="destination",
                             description="What is the destination of the trip?")
travel_date_schema = ResponseSchema(name="travel_date",
                                      description="When is the trip planned for?")
requests_schema = ResponseSchema(name="requests",
                                    description="""Extract any specific requests made by the traveler,
                                    and output them as a comma separated Python list.""")
budget_schema = ResponseSchema(name="budget",
                             description="""What is the budget for the entire trip? 
                                If this information is not found, output -1.""")

completion_schemas = [destination_schema, 
                    travel_date_schema,
                    requests_schema,
                    budget_schema]
output_parser = StructuredOutputParser.from_response_schemas(completion_schemas)        
# output_parser
format_instructions = output_parser.get_format_instructions()            
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"destination": string  // What is the destination of the trip?
	"travel_date": string  // When is the trip planned for?
	"requests": string  // Extract any specific requests made by the traveler,
                                    and output them as a comma separated Python list.
	"budget": string  // What is the budget for the entire trip? 
                                If this information is not found, output -1.
}
```


In [6]:
vacation_template = """\
For the following query, extract the following information:

destination: What is the destination of the trip?\

travel_date: When is the trip planned for?\

requests: Extract any specific requests made by the traveler,\
and output them as a comma separated Python list.

budget: What is the budget for the entire trip? \
If this information is not found, output -1.

Format the output as JSON with the following keys:
destination
travel_date
requests
budget

query: {query}

{format_instructions}
"""
prompt_template = ChatPromptTemplate.from_template(vacation_template)
prompt_template.pretty_print()


================================ Human Message =================================

For the following query, extract the following information:

destination: What is the destination of the trip?
travel_date: When is the trip planned for?
requests: Extract any specific requests made by the traveler,and output them as a comma separated Python list.

budget: What is the budget for the entire trip? If this information is not found, output -1.

Format the output as JSON with the following keys:
destination
travel_date
requests
budget

query: {query}

{format_instructions}



In [37]:
prompt = prompt_template.format_messages(
    query = customer_query,
    format_instructions = format_instructions)
print(prompt)
print("The formated prompt is:\n")
nprint(prompt[0].content)

[HumanMessage(content='For the following query, extract the following information:\n\ndestination: What is the destination of the trip?\ntravel_date: When is the trip planned for?\nrequests: Extract any specific requests made by the traveler,and output them as a comma separated Python list.\n\nbudget: What is the budget for the entire trip? If this information is not found, output -1.\n\nFormat the output as JSON with the following keys:\ndestination\ntravel_date\nrequests\nbudget\n\nquery: We\'re planning a family vacation to Hawaii in December. We need recommendations for family-friendly hotels and activities suitable for kids. Also, any travel advisories we should be aware of?Our budget for the entire trip is around $5000.\n\n\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"destination": string  // What is the destination of the trip?\n\t"travel_date": string  // When is the tr

In [38]:
completion = QAchat(prompt)
print(completion.content)

```json
{
	"destination": "Hawaii",
	"travel_date": "December",
	"requests": "family-friendly hotels, activities suitable for kids, travel advisories",
	"budget": "$5000"
}
```
